## Deliverable 2. Create a Customer Travel Destinations Map.

In [42]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [43]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Tiksi,RU,71.6872,128.8694,39.54,90,99,7.58,overcast clouds
1,1,Qaanaaq,GL,77.4840,-69.3632,49.44,65,90,8.77,overcast clouds
2,2,Prince Rupert,CA,54.3161,-130.3201,55.49,96,100,11.50,light rain
3,3,Tahta,EG,26.7693,31.5021,86.86,25,0,10.67,clear sky
4,4,Olafsvik,IS,64.8945,-23.7142,51.57,84,100,14.76,overcast clouds


In [44]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min = float(input('What is the minimum temperature for your vacation?'))
max = float(input('What is the maximum temperature for your vacation?'))

In [45]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
city_data_df = city_data_df.loc[city_data_df['Max Temp'] > min]
city_data_df = city_data_df.loc[city_data_df['Max Temp'] < max]
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Tahta,EG,26.7693,31.5021,86.86,25,0,10.67,clear sky
8,8,Bathsheba,BB,13.2167,-59.5167,86.61,86,75,19.57,broken clouds
37,37,Ciudad Bolivar,VE,8.1222,-63.5497,85.69,64,88,3.24,overcast clouds
63,63,Albany,US,42.6001,-73.9662,86.58,35,7,2.44,clear sky
104,104,Bethel,US,41.3712,-73.4140,87.19,46,41,1.99,scattered clouds


In [46]:
# 4a. Determine if there are any empty rows.
city_data_df.count()

City_ID                27
City                   27
Country                27
Lat                    27
Lng                    27
Max Temp               27
Humidity               27
Cloudiness             27
Wind Speed             27
Current Description    27
dtype: int64

In [47]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_hotel_df = city_data_df.dropna()

In [48]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_hotel_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Tahta,EG,86.86,clear sky,26.7693,31.5021,
8,Bathsheba,BB,86.61,broken clouds,13.2167,-59.5167,
37,Ciudad Bolivar,VE,85.69,overcast clouds,8.1222,-63.5497,
63,Albany,US,86.58,clear sky,42.6001,-73.9662,
104,Bethel,US,87.19,scattered clouds,41.3712,-73.4140,
114,Victoria,HK,86.38,overcast clouds,22.2855,114.1577,
144,Laramie,US,85.08,broken clouds,41.3114,-105.5911,
200,Nalut,LY,86.43,clear sky,30.3333,10.8500,
214,Acapulco,MX,86.81,overcast clouds,16.8634,-99.8901,
257,San Lorenzo,PY,86.81,light rain,-25.3333,-57.5333,


In [49]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    request = requests.get(base_url, params=params).json()

    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = request["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
    
hotel_df
    
        

Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Tahta,EG,86.86,clear sky,26.7693,31.5021,فندق الزيات
8,Bathsheba,BB,86.61,broken clouds,13.2167,-59.5167,Atlantis Hotel
37,Ciudad Bolivar,VE,85.69,overcast clouds,8.1222,-63.5497,Hotel Edi
63,Albany,US,86.58,clear sky,42.6001,-73.9662,
104,Bethel,US,87.19,scattered clouds,41.3712,-73.4140,Courtyard by Marriott Danbury
114,Victoria,HK,86.38,overcast clouds,22.2855,114.1577,Mini Central
144,Laramie,US,85.08,broken clouds,41.3114,-105.5911,Fairfield Inn & Suites by Marriott Laramie
200,Nalut,LY,86.43,clear sky,30.3333,10.8500,
214,Acapulco,MX,86.81,overcast clouds,16.8634,-99.8901,HS Hotsson Smart Acapulco
257,San Lorenzo,PY,86.81,light rain,-25.3333,-57.5333,Canarias Hotel


In [50]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df['Hotel Name']!= '']
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Tahta,EG,86.86,clear sky,26.7693,31.5021,فندق الزيات
8,Bathsheba,BB,86.61,broken clouds,13.2167,-59.5167,Atlantis Hotel
37,Ciudad Bolivar,VE,85.69,overcast clouds,8.1222,-63.5497,Hotel Edi
104,Bethel,US,87.19,scattered clouds,41.3712,-73.4140,Courtyard by Marriott Danbury
114,Victoria,HK,86.38,overcast clouds,22.2855,114.1577,Mini Central
144,Laramie,US,85.08,broken clouds,41.3114,-105.5911,Fairfield Inn & Suites by Marriott Laramie
214,Acapulco,MX,86.81,overcast clouds,16.8634,-99.8901,HS Hotsson Smart Acapulco
257,San Lorenzo,PY,86.81,light rain,-25.3333,-57.5333,Canarias Hotel
322,Kahului,US,85.96,few clouds,20.8947,-156.4700,Maui Seaside Hotel
330,Dalbandin,PK,87.44,overcast clouds,28.8947,64.4101,Sanjrani House


In [51]:
# 8a. Create the output File (CSV)
output = "Vacation_Search/WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output, index_label="City_ID")

In [55]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [59]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)

# 11b. Display the figure

fig

Figure(layout=FigureLayout(height='420px'))